## Import

In [2]:
import json
from tqdm import tqdm
from http import HTTPStatus

In [3]:
import dashscope
dashscope.api_key = "sk-xxxxx"

In [4]:
instruction = '''以下是关于医学知识的单项选择题，请根据题目输出唯一的答案选项\n'''
instruction

'以下是关于医学知识的单项选择题，请根据题目输出唯一的答案选项\n'

In [5]:
path_file_data = '../data/MedQA/Mainland/test.jsonl'

## Data

In [6]:
list_dict_test = []
with open(path_file_data, 'r', encoding="utf-8") as f:
    for idx, line in enumerate(f):
        data = json.loads(line)
        data['ID'] = idx
        data['A'], data['B'], data['C'], data['D'], data['E'] = data['options']['A'], data['options']['B'], data['options']['C'], data['options']['D'], data['options']['E']
        del data['options']
        list_dict_test.append(data)

In [7]:
list_dict_test[0]

{'question': '经调查证实出现医院感染流行时，医院应报告当地卫生行政部门的时间是（\u3000\u3000）。',
 'answer': '24小时内',
 'meta_info': '卫生法规',
 'answer_idx': 'E',
 'ID': 0,
 'A': '2小时',
 'B': '4小时内',
 'C': '8小时内',
 'D': '12小时内',
 'E': '24小时内'}

In [8]:
for idx, dict_test in enumerate(tqdm(list_dict_test)):
    question, answer = dict_test['question'], dict_test['answer']
    a, b, c, d, e = dict_test['A'], dict_test['B'], dict_test['C'], dict_test['D'], dict_test['E']
    question = question.replace("（ ）。", "")
    input = instruction + f"问题：{question}: (A){a}, (B){b}, (C){c}, (D){d}, (E){e}\n" + "答案："
    dict_test['Input'] = input
    # print(input, '\t' ,answer)

100%|██████████| 3426/3426 [00:00<00:00, 685184.32it/s]


## Test

In [23]:
# config of model
model = "qwen-72b-chat"
# qwen1.5-72b-chat, qwen1.5-14b-chat, qwen1.5-7b-chat
# qwen-72b-chat, qwen-14b-chat, qwen-7b-chat
# default config of model
seed = 1234
max_tokens = 1500
# repetition_penalty  = 1.1
result_format = 'text'
# seted by user
top_p = 0.0000001
top_k = 0
temperature = 0
repetition_penalty  = 0

In [24]:
def get_response(input, flag_print=True):
    response = dashscope.Generation.call(
        model=model,
        messages=[
            {"role": "user", "content": input}
        ],
        result_format=result_format,
        top_p=top_p,
        top_k=top_k,
        temperature=temperature,
    )
    if not response.status_code == HTTPStatus.OK:
        print(
            "Request id: %s, Status code: %s, error code: %s, error message: %s"
            % (
                response.request_id,
                response.status_code,
                response.code,
                response.message,
            )
        )
    if flag_print:
        print(f'Response: {response["output"]["text"]}', )
        print(f"Finish_reason: {response['output']['finish_reason']}")
        print(response['usage'])
    return response

In [30]:
idx = 6
input = list_dict_test[idx]['Input']
print(input)
response = get_response(input)
print('--------------------------------------------')
print(f"Ground-truth: {list_dict_test[idx]['answer']}")

以下是关于医学知识的单项选择题，请根据题目输出唯一的答案选项
问题：胃癌最主要的转移途径是（　　）。: (A)直接蔓延, (B)淋巴转移, (C)血行转移, (D)腹腔内种植, (E)盆腔转移
答案：
Response: (B)淋巴转移
Finish_reason: stop
{"input_tokens": 65, "output_tokens": 4, "total_tokens": 69}
--------------------------------------------
Ground-truth: 淋巴转移


## Task

In [31]:
name_task = f"QWEN72B-Prompt-{model}"

In [ ]:
with open(f'../result/{name_task}.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(list_dict_test, indent=2, ensure_ascii=False))

In [ ]:
list_dict_test[0]

## Run

In [ ]:
name_task

In [ ]:
start_idx = 0
list_failed = []
for idx, dict_test in enumerate(tqdm(list_dict_test[start_idx:])):
    idx = start_idx + idx
    input = dict_test['Input']
    try:
        response = get_response(input, flag_print=False)
        dict_test['Result'] = response['output']['text']
        dict_test['finish_reason'] = response['output']['finish_reason']
        dict_test['usage'] = response['usage']
    except:
        print(f"Failed idx: {idx}")
        list_failed.append(idx)

In [ ]:
list_failed_failed = []
for idx_failed in tqdm(list_failed):
    dict_test = list_dict_test[idx_failed]
    input = dict_test['Input']
    try:
        response = get_response(input, flag_print=False)
        dict_test['Result'] = response['output']['text']
        dict_test['finish_reason'] = response['output']['finish_reason']
        dict_test['usage'] = response['usage']
    except:
        print(f"Failed idx: {idx_failed}")
        list_failed_failed.append(idx_failed)

In [ ]:
for idx_failed in list_failed_failed:
    list_dict_test[idx_failed]['Result'] = 'None'

In [ ]:
with open(f"../result/{name_task}.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(list_dict_test, indent=2, ensure_ascii=False))

## Performance

### Read

In [ ]:
with open(f'../result/{name_task}.json', 'r', encoding="utf-8") as f:
    list_dict_test = json.load(f)

In [ ]:
list_dict_test[0]

In [ ]:
len(list_dict_test), len([dict_test for dict_test in list_dict_test if dict_test.get('usage')])

### Prediction

In [ ]:
def get_prediction(result_pred, list_option_name):
    list_str_split = ['最终答案是','最终答案为','最可能的选项是','最可能的诊断是','正确的选项是', '正确选项是', '正确选项为','最可能的答案是','最有可能的答案是']
    for str_split in list_str_split:
        if len(result_pred.split(str_split))>1:
            result_pred = result_pred.split(str_split)[-1]
            result_pred = result_pred.split('。')[0]
            break
    list_option_idx = [ chr(ord('A') + idx) for idx in range(len(list_option_name))]
    list_option_idx_name = [ [option_idx, option_name] for option_idx, option_name in zip(list_option_idx, list_option_name)]
    list_option_idx_name = sorted(list_option_idx_name, key=lambda x: len(x[1]), reverse=True)
    list_option_match = []
    # search option_idx and option_name
    for option_idx, option_name in list_option_idx_name:
        if option_idx in result_pred or option_name in result_pred:
            list_option_match.append(option_idx)
            if option_name in result_pred:
                result_pred = result_pred.replace(option_name, '')
    if len(list_option_match)==1:
        return list_option_match[0]
    elif len(list_option_match)>1:
        return None
    else:
        return None

In [ ]:
list_dict_test[1]

In [ ]:
list_dict_result_correct, list_dict_result_wrong, list_dict_result_none = [], [], []
for idx, dict_test in enumerate(tqdm(list_dict_test)):
    list_option_name = [dict_test['A'], dict_test['B'], dict_test['C'], dict_test['D'], dict_test['E']]
    dict_test['prediction'] = get_prediction(dict_test['Result'], list_option_name)
    if not dict_test['prediction']:
        list_dict_result_none.append(dict_test)
    elif dict_test['answer'] == dict_test['prediction']:
        list_dict_result_correct.append(dict_test)
    else:
        list_dict_result_wrong.append(dict_test)
    # if len(dict_test['Result']) > 6:
    #     print(dict_test['Result']) 

In [ ]:
count_correct = len(list_dict_result_correct)
count_wrong = len(list_dict_result_wrong)
count_none = len(list_dict_result_none)
acc = round(len(list_dict_result_correct) / len(list_dict_test)*100, 2)
print(f"Accuracy is {acc}% with {count_correct} correct, {count_wrong} wrong, and {count_none} none")

In [ ]:
# Too long 111

In [ ]:
with open(f'../result/{name_task}.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(list_dict_test, indent=4, ensure_ascii=False))

### Accuracy for different types of questions

In [ ]:
with open(f'../result/{name_task}-SAVE.json', 'r', encoding="utf-8") as f:
    list_dict_test_sample = json.load(f)
with open('../data/list_test_knowledge_idx.txt', 'r') as f:
    list_test_knowledge_idx = f.readlines()
with open('../data/list_test_example_idx.txt', 'r') as f:
    list_test_example_idx = f.readlines()

In [ ]:
list_test_knowledge_idx = [ int(idx.strip()) for idx in list_test_knowledge_idx if idx.strip() ]
list_test_example_idx = [ int(idx.strip()) for idx in list_test_example_idx if idx.strip() ]
list_dict_test_knowledge = [ list_dict_test_sample[idx] for idx in list_test_knowledge_idx ]
list_dict_test_example = [ list_dict_test_sample[idx] for idx in list_test_example_idx ]
print(len(list_dict_test_sample), len(list_dict_test_knowledge), len(list_dict_test_example))

In [ ]:
def cal_performance(list_dict_test_sample):
    list_dict_result_correct, list_dict_result_wrong, list_dict_result_none = [], [], []
    for idx, dict_test in enumerate(list_dict_test_sample):
        if not dict_test['prediction']:
            list_dict_result_none.append(dict_test)
        elif dict_test['answer'] == dict_test['prediction']:
            list_dict_result_correct.append(dict_test)
        else:
            list_dict_result_wrong.append(dict_test)
    count_correct = len(list_dict_result_correct)
    count_wrong = len(list_dict_result_wrong)
    count_none = len(list_dict_result_none)
    acc = len(list_dict_result_correct) / len(list_dict_test_sample)
    print(f"Accuracy is {round(acc*100, 2)}% with {count_none} none, {count_correct} correct, and {count_wrong} wrong， total {len(list_dict_test_sample)}")
    return round(acc, 4), count_none, count_correct, count_wrong

In [ ]:
acc_MK, count_none_MK, _, _ = cal_performance(list_dict_test_knowledge)
acc_CA, count_none_CA, _, _ = cal_performance(list_dict_test_example)
acc_all, count_none_all, _, _ = cal_performance(list_dict_test_sample)

In [ ]:
print(count_none_MK, acc_MK, count_none_CA, acc_CA, count_none_all, acc_all, sep='\t')

## End

In [ ]:
print('Done')